In [24]:
import numpy as np
import math
import pandas as pd
import hashlib, random
from sklearn.mixture import GaussianMixture
import copy
import itertools
import matplotlib.pyplot as plt
from collections import defaultdict
import seaborn as sns
import jsonpickle
from random import shuffle
from collections import defaultdict
import os, json
from datetime import datetime
import matplotlib.ticker as mticker
from matplotlib.ticker import MaxNLocator
pd.options.display.float_format = '{:,.2f}'.format
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import random, json
from tqdm import trange, tqdm
from operator import itemgetter
from analyzer import *
import query_gen

sns.set(style='ticks', palette='coolwarm') #this overwrites matplotlib setting so make it run first 

# def set_font():
import matplotlib
# http://ishxiao.com/blog/python/2017/07/23/how-to-change-the-font-size-on-a-matplotlib-plot.html

font = {'weight' : 'bold',
#         'size'   : 20,
       'family': 'Linux Libertine O'}
matplotlib.rc('font', **font)

In [25]:
def analyze_ratio(results):
    cnt_mean_ratios = {}
    sum_mean_ratios = {}
    cnt_median_ratios = {}
    sum_median_ratios = {}
    cnt_ratios = {}
    sum_ratios = {}
    gt = results['GT']
    for name, result in results.items():
        if name == 'GT':
            continue
#         print(name)
#         print(gt)
#         print(result)
        cnt_ratio = [float(b[0])/float(g[0]) for b, g in zip(result, gt)]
        cnt_mean_ratios[name] = np.mean(cnt_ratio)
        cnt_median_ratios[name] = np.median(cnt_ratio)
#         print(pd.DataFrame(cnt_ratio).describe())
        sum_ratio = [float(b[1])/float(g[1]) for b, g in zip(result, gt)]
        sum_mean_ratios[name] = np.mean(sum_ratio)
        sum_median_ratios[name] = np.median(sum_ratio)
        sum_ratios[name] = sum_ratio
        cnt_ratios[name] = cnt_ratio
    return cnt_ratios, sum_ratios, cnt_mean_ratios, cnt_median_ratios, sum_mean_ratios, sum_median_ratios

def plot_columns(mean, median, ylabel='Bound/GroundTruth Ratio', 
                 xlabel='Mean and Median', bar_labels=['Mean Ratio', 'Median Ratio'],
                log_scale = True, yrange= None):
    patterns = [ "/" , "\\" , "|" , "-" , "+" , "x", "o", "O", ".", "*" ]
    baselines = mean.keys()
#     print(baselines)
    if yrange is None:
        yrange = [0, max(max(mean.values()), max(median.values())) + 1]
    fig, ax = plt.subplots(1,1,figsize=(18,8))
    
    xfont = {'weight' : 'bold',
            'size'   : 22}

    width = 0.3
    
    mean_bar = [perf for perf in mean.values()]
    median_bar = [perf for perf in median.values()]

    r0 = np.arange(len(mean_bar))
    r1 = [x - 0.15 for x in r0]
    r2 = [x + width for x in r1]
#     print(mean_bar, median_bar)
    ax.bar(r1, mean_bar, width, edgecolor='black', label=bar_labels[0], hatch=patterns[0])
    ax.bar(r2, median_bar, width, edgecolor='black', label=bar_labels[1], hatch=patterns[1])
    ax.set_xticks(np.arange(len(baselines)))
    ax.set_xticklabels(baselines, fontsize='x-large')
    
    ax.hlines(1.0, -0.5, len(baselines)-0.5, linestyle='--', color='gray', linewidth=4, label='Ground Truth')
    
    ax.set_ylabel(ylabel, fontdict = xfont)
    
    ax.set_ylim(yrange)
    
    if log_scale:
        ax.set_yscale('log')
        
    ax.legend()
    
#     matplotlib.pyplot.sca(ax)
    print(yrange)

    plt.legend(fontsize=20, loc='upper left')
    ax.set_xlabel(xlabel, fontdict = xfont)
    fig.tight_layout()
    plt.show()

    
# https://seaborn.pydata.org/tutorial/color_palettes.html
def plot_box(cnt_ratios, sum_ratios, ylabel='Bound/GroundTruth Ratio', xlabel='Mean and Median', bar_labels=['COUNT(*)', 'SUM(Light)']):
    patterns = [ "/" , "\\" , "|" , "-" , "+" , "x", "o", "O", ".", "*" ]
    baselines = cnt_ratios.keys()
#     print(max([max(r) for r in cnt_ratios.values()]), max(cnt_ratios.values()))
#     yrange = [0, max(max([max(r) for r in cnt_ratios.values()]), max([max(r) for r in sum_ratios.values()])) + 1]
    fig, ax = plt.subplots(1,1,figsize=(18,8))
    xfont = {'weight' : 'bold',
            'size'   : 22}

    width = 0.5
    
    cnt_box = [perf for perf in cnt_ratios.values()]
    sum_box = [perf for perf in sum_ratios.values()]

    r0 = np.arange(len(cnt_ratios))
    r1 = [x - 0.15 for x in r0]
    r2 = [x + width for x in r1]
#     print(mean_bar, median_bar)
    ax.boxplot(cnt_box, positions = r1, showfliers=False)
    ax.boxplot(sum_box, positions = r2, showfliers=False)
    ax.set_xticks(np.arange(len(baselines)+1))
    ax.set_xticklabels(baselines, fontsize='x-large')

    ax.hlines(1.0, -1, len(baselines)+1.5, linestyle='--', color='gray', linewidth=2, label='Ground Truth')

    ax.set_ylabel(ylabel, fontdict = xfont)
    ax.set_xlim([-1, len(baselines)+1])
#     ax.set_ylim(yrange)
#         ax.set_yscale("log")

    ax.legend()

    #     matplotlib.pyplot.sca(ax)

#         plt.legend(fontsize=20, loc='upper left')
#     ax.set_xlabel(xlabel, fontdict = xfont)
    fig.tight_layout()
    plt.show()

In [26]:


def restore_obj(path):
    try:
        with open(path) as f:
            s = f.read()
            data = jsonpickle.decode(s, keys = True)
            print("Loaded %s" % path)
            return data
    except Exception as e:
        print("failed to restore", path, e)
        return None

def load_and_merge(config):
    # load all the files and merge them together because we have different files representing different partition type.
    baseline_perf = defaultdict(dict)
    queries = None
    for name, path in config.items():
        path = '/local/xi/VarAcc/src/cache/'+path
        if 'json' in path:
            data = json.load(open(path))
        else:
            data = restore_obj(path)
        if name == 'GT':
            queries = data
        else:
            k = name.split('-')[0]
            for base, perf in data.items():
                baseline_perf[k][base] = perf
    return queries, baseline_perf


In [28]:
!hostname

vulture


In [23]:
ds_name = "IntelWireless"

n_queries = {'random':2000}
gts = {'random':'IntelWireless-RangeSumQuery-itime-light-2000.jpk'}
 
k_depths =[
          [7,64]
        ]

baselines=[
        {'alias':"PASS", 'name':"DP_AMAX"}, 
        {'alias':"US", 'name':"EqualDepthPartitioner"}, 
        {'alias':"ST", 'name':"EqualDepthPartitioner"}, 
        {'alias':"AQP++", "name":'HillClimbing'},
     ]

name_temp = "{sr}-{alias}"
path_temp = "{ds}-{name}-{alias}-{depth}-{k}-{sr}-{n_query}-{qname}.json"

config={}

srs=[0.005]

k_depths =[
    
    [7,64], 
    
        ]

name_temp = "{k}-{alias}"


for workload, n_query in n_queries.items():
    workload_config={}
    workload_config['GT']=gts[workload]
    for kd in k_depths:
        k=kd[1]
        depth=kd[0]
        for sr in srs:
            for baseline in baselines:
                    alias=baseline['alias']
                    name=baseline['name']
                    baseline_name = name_temp.format(k=k, alias=alias)
                    baseline_path = path_temp.format(ds=ds_name, name=name, alias=alias, depth=depth, k=k, sr=sr,n_query=n_query, qname=workload)
                    workload_config[baseline_name] = baseline_path
    config[workload]=workload_config


queries, bp = load_and_merge(config['random'])
random_analyzed = defaultdict(dict)
for k in bp.keys():
    # SUM query only
    RE_results = RelativeError(queries, bp[k]).analyze()
    FR_results = FailureRate(queries, bp[k]).analyze()
    CI_results = PERCENTILE_CI(queries, bp[k]).analyze()
    CIR_results = CIRatio(queries, bp[k]).analyze()
    query_type = 1 #0 for cnt, 1 for sum, 2 for avg
    random_analyzed[k]={"Relative Error": RE_results[query_type], 
                        "Failure Rate": FR_results[query_type], 
                        "Half CI": CI_results[query_type], "CI Ratio": CIR_results[query_type]
                       }

by_q_type  = {}
for qname, query_type in {"CNT": 0, "SUM": 1, "AVG": 2, "MIN": 3, "MAX": 4}.items():
    for k in bp.keys():
        # SUM query only
        RE_results = RE_w_MinMax(queries, bp[k]).analyze()
#         FR_results = FailureRate(queries, bp[k]).analyze()
#         CI_results = PERCENTILE_CI(queries, bp[k]).analyze()
#         CIR_results = CIRatio(queries, bp[k]).analyze()
        by_q_type[k]={"Relative Error": RE_results[query_type], 
#                             "Failure Rate": FR_results[query_type], 
#                             "Half CI": CI_results[query_type], "CI Ratio": CIR_results[query_type]
                           }
    print(qname)
    for base in ['PASS', 'US', 'ST', 'AQP++']:
        print(base, np.percentile(by_q_type['64']['Relative Error'][base], 50))

Loaded /local/xi/VarAcc/src/cache/IntelWireless-RangeSumQuery-itime-light-2000.jpk
CNT
PASS 0.000992317698090024
US 0.009448071478984263
ST 0.0016109560952192372
AQP++ 0.003353882644568703
SUM
PASS 0.0007895222109930471
US 0.016145269030771042
ST 0.010484856069597325
AQP++ 0.005018299861210685
AVG
PASS 0.0012000180121661311
US 0.012243917644156867
ST 0.010524939740060522
AQP++ 0.004033928800972551
MIN
PASS 0.0
US 522.2189796547437
ST 522.2189796547437
AQP++ 0.0
MAX
PASS 0.0
US 0.0
ST 0.0
AQP++ 0.0


In [6]:
ds_name = "Instacart"

n_queries = {'random':2000}
gts = {'random':'Instacart-RangeSumQuery-product_id-reordered-2000.jpk'}
     
baselines=[
        {'alias':"PASS", 'name':"DP_AMAX"}, 
        {'alias':"US", 'name':"EqualDepthPartitioner"}, 
        {'alias':"ST", 'name':"EqualDepthPartitioner"}, 
        {'alias':"AQP++", "name":'HillClimbing'},
     ]
config={}

srs=[0.005]

k_depths =[
    
    [7,64], 
    
        ]

name_temp = "{k}-{alias}"
path_temp = "{ds}-{name}-{alias}-{depth}-{k}-{sr}-{n_query}-{qname}.json"

for workload, n_query in n_queries.items():
    workload_config={}
    workload_config['GT']=gts[workload]
    for kd in k_depths:
        k=kd[1]
        depth=kd[0]
        for sr in srs:
            for baseline in baselines:
                    alias=baseline['alias']
                    name=baseline['name']
                    baseline_name = name_temp.format(k=k, alias=alias)
                    baseline_path = path_temp.format(ds=ds_name, name=name, alias=alias, depth=depth, k=k, sr=sr,n_query=n_query, qname=workload)
                    workload_config[baseline_name] = baseline_path
    config[workload]=workload_config


queries, bp = load_and_merge(config['random'])
random_analyzed = defaultdict(dict)
for k in bp.keys():
    # SUM query only
    RE_results = RelativeError(queries, bp[k]).analyze()
    FR_results = FailureRate(queries, bp[k]).analyze()
    CI_results = PERCENTILE_CI(queries, bp[k]).analyze()
    CIR_results = CIRatio(queries, bp[k]).analyze()
    query_type = 1 #0 for cnt, 1 for sum, 2 for avg
    random_analyzed[k]={"Relative Error": RE_results[query_type], 
                        "Failure Rate": FR_results[query_type], 
                        "Half CI": CI_results[query_type], "CI Ratio": CIR_results[query_type]
                       }

by_q_type  = {}
for qname, query_type in {"CNT": 0, "SUM": 1, "AVG": 2, "MIN": 3, "MAX": 4}.items():
    for k in bp.keys():
        # SUM query only
        RE_results = RE_w_MinMax(queries, bp[k]).analyze()
#         FR_results = FailureRate(queries, bp[k]).analyze()
#         CI_results = PERCENTILE_CI(queries, bp[k]).analyze()
#         CIR_results = CIRatio(queries, bp[k]).analyze()
        by_q_type[k]={"Relative Error": RE_results[query_type], 
#                             "Failure Rate": FR_results[query_type], 
#                             "Half CI": CI_results[query_type], "CI Ratio": CIR_results[query_type]
                           }
    print(qname)
    for base in ['PASS', 'US', 'ST', 'AQP++']:
        print(base, np.percentile(by_q_type['64']['Relative Error'][base], 50))

Loaded /local/xi/VarAcc/src/cache/Instacart-RangeSumQuery-product_id-reordered-2000.jpk
CNT
PASS 0.0003697714051830369
US 0.012057285011092708
ST 0.0022665251770069856
AQP++ 0.0037569305156833713
SUM
PASS 0.0006300513150947784
US 0.018237932228759454
ST 0.01277132521037442
AQP++ 0.004738916755839947
AVG
PASS 0.000534980015427974
US 0.01254999638680509
ST 0.012258833351829828
AQP++ 0.003053937363282077
MIN
PASS 0.0
US 0.0
ST 0.0
AQP++ 0.0
MAX
PASS 0.0
US 0.0
ST 0.0
AQP++ 0.0


In [11]:

ds_name = "Taxi"

n_queries = {'random':2000}
gts = {'random':'Taxi-RangeSumQuery-pickup_datetime-trip_distance-2000.jpk'}
          
baselines=[
        {'alias':"PASS", 'name':"DP_AMAX"}, 
        {'alias':"US", 'name':"EqualDepthPartitioner"}, 
        {'alias':"ST", 'name':"EqualDepthPartitioner"}, 
        {'alias':"AQP++", "name":'HillClimbing'},
     ]
config={}

srs=[0.005]

k_depths =[
    
    [7,64], 
    
        ]

name_temp = "{k}-{alias}"
path_temp = "{ds}-{name}-{alias}-{depth}-{k}-{sr}-{n_query}-{qname}.json"

for workload, n_query in n_queries.items():
    workload_config={}
    workload_config['GT']=gts[workload]
    for kd in k_depths:
        k=kd[1]
        depth=kd[0]
        for sr in srs:
            for baseline in baselines:
                    alias=baseline['alias']
                    name=baseline['name']
                    baseline_name = name_temp.format(k=k, alias=alias)
                    baseline_path = path_temp.format(ds=ds_name, name=name, alias=alias, depth=depth, k=k, sr=sr,n_query=n_query, qname=workload)
                    workload_config[baseline_name] = baseline_path
    config[workload]=workload_config


queries, bp = load_and_merge(config['random'])
random_analyzed = defaultdict(dict)
for k in bp.keys():
    # SUM query only
    RE_results = RelativeError(queries, bp[k]).analyze()
    FR_results = FailureRate(queries, bp[k]).analyze()
    CI_results = PERCENTILE_CI(queries, bp[k]).analyze()
    CIR_results = CIRatio(queries, bp[k]).analyze()
    query_type = 1 #0 for cnt, 1 for sum, 2 for avg
    random_analyzed[k]={"Relative Error": RE_results[query_type], 
                        "Failure Rate": FR_results[query_type], 
                        "Half CI": CI_results[query_type], "CI Ratio": CIR_results[query_type]
                       }

by_q_type  = {}
for qname, query_type in {"CNT": 0, "SUM": 1, "AVG": 2, "MIN": 3, "MAX": 4}.items():
    for k in bp.keys():
        # SUM query only
        RE_results = RE_w_MinMax(queries, bp[k]).analyze()
#         FR_results = FailureRate(queries, bp[k]).analyze()
#         CI_results = PERCENTILE_CI(queries, bp[k]).analyze()
#         CIR_results = CIRatio(queries, bp[k]).analyze()
        by_q_type[k]={"Relative Error": RE_results[query_type], 
#                             "Failure Rate": FR_results[query_type], 
#                             "Half CI": CI_results[query_type], "CI Ratio": CIR_results[query_type]
                           }
    print(qname)
    for base in ['PASS', 'US', 'ST', 'AQP++']:
        print(base, np.percentile(by_q_type['64']['Relative Error'][base], 50))

Loaded /local/xi/VarAcc/src/cache/Taxi-RangeSumQuery-pickup_datetime-trip_distance-2000.jpk
CNT
PASS 0.00023983516212723453
US 0.005080087997217773
ST 0.0008757561812368392
AQP++ 0.0016471239849696106
SUM
PASS 0.0005618153333024449
US 0.010002918915751634
ST 0.008876689228049264
AQP++ 0.0027018772314121066
AVG
PASS 0.0005245173090414003
US 0.0087711064083602
ST 0.008904518018372169
AQP++ 0.0022275934062794918
MIN
PASS 0.0
US 0.0
ST 0.0
AQP++ 0.0
MAX
PASS 0.0
US 0.9030540887683745
ST 0.9047367155566242
AQP++ 0.0
